In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U -qq git+https://github.com/huggingface/diffusers.git
!pip install -qq accelerate tensorboard transformers ftfy gradio
!pip install -qq "ipywidgets>=7,<8"
!pip install -qq bitsandbytes


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 835.3 kB/s eta

In [ ]:
!pip install -U --pre triton

from subprocess import getoutput
import time

s = getoutput('nvidia-smi')
if 'T4' in s:
    gpu = 'T4'
elif 'P100' in s:
    gpu = 'P100'
elif 'V100' in s:
    gpu = 'V100'
elif 'A100' in s:
    gpu = 'A100'
else:
    gpu = None
    print("Your GPU may not be supported for xformers precompiled wheels.")

if gpu == 'T4':
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
elif gpu == 'P100':
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl
elif gpu == 'V100':
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl
elif gpu == 'A100':
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl


Your GPU may not be supported for xformers precompiled wheels.


In [ ]:
import os
import math
import random
import itertools
import gc
from pathlib import Path

import torch
import torch.nn.functional as F
import torch.utils.data
from torch.utils.data import Dataset

import numpy as np
from PIL import Image
from tqdm.auto import tqdm

from accelerate import Accelerator, notebook_launcher, logging as accel_logging, utils as accel_utils
from accelerate.utils import set_seed

from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    StableDiffusionPipeline,
    UNet2DConditionModel,
)
from diffusers.optimization import get_scheduler

from transformers import CLIPTextModel, CLIPTokenizer

import bitsandbytes as bnb


ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

In [ ]:
!dir "/content/drive/MyDrive/CS614 Final Project/Dataset/Milan"

IMG_20230423_082747.jpg    IMG_20240528_105934.jpg  IMG_20240903_062903.jpg
IMG_20230423_083415.jpg    IMG_20240531_014610.jpg  IMG_20240911_063611.jpg
IMG_20230901_095400.jpg    IMG_20240712_092230.jpg  IMG_20240911_063615.jpg
IMG_20230901_095403.jpg    IMG_20240712_092238.jpg  IMG_20240911_063636.jpg
IMG_20231101_172932.jpg    IMG_20240712_092244.jpg  IMG_20240911_063732.jpg
IMG_20240303_070457~3.jpg  IMG_20240825_113037.jpg  IMG_20240911_063840.jpg
IMG_20240308_072035.jpg    IMG_20240901_163621.jpg
IMG_20240528_105928.jpg    IMG_20240901_163641.jpg


In [ ]:
# The pretrained Stable Diffusion model to start from
pretrained_model_name_or_path = r"/content/drive/MyDrive/CS614 Final Project/model/"

# Unique prompt identifier for your pet dog Bruno
instance_prompt = "<bruno> a photo of my pet dog Bruno"

# Path to your custom dataset of Bruno's images (ensure this folder exists)
instance_data_dir = r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/"

# Option for prior preservation (set to False if you only train on Bruno)
with_prior_preservation = False
prior_loss_weight = 1.0  # adjust if using prior preservation

# If using prior preservation, you can specify a class prompt and folder for generic dog images:
class_prompt = "a photo of a dog"
class_data_dir = r"/content/drive/MyDrive/CS614 Final Project/class_images"  # make sure this folder exists (or let the code create it)
num_class_images = 12  # number of class images to generate/use

# Training hyperparameters
learning_rate = 5e-6
max_train_steps = 300  # adjust based on dataset size and experimentation
train_batch_size = 2
gradient_accumulation_steps = 2
max_grad_norm = 1.0

# Mixed precision and gradient checkpointing settings
mixed_precision = "fp16"  # or "bf16" if supported
gradient_checkpointing = True
use_8bit_adam = True

# Output directory to save checkpoints and the final model
output_dir = "dreambooth-bruno"
seed = 3434554


In [ ]:
# Captions for images in your dataset
captions = {
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20231101_172932.jpg": "<bruno> sitting happily with his tongue out, looking cheerful and excited. The bright expression on his face and his slightly tilted head give him an adorable and friendly vibe.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20230423_082747.jpg": "<bruno> lying comfortably on a green platform, looking up with his tongue out in a joyful and relaxed manner. His soft golden fur contrasts beautifully with the background.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20230423_083415.jpg": "<bruno> sitting inside his playpen, full of energy and excitement. His bright eyes and wagging tail suggest he is happy and eager for attention.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20230901_095400.jpg": "<bruno> lying on the ground with a big happy grin, enjoying the moment as he gets a gentle pat on his chin. His playful and affectionate nature shines through in this close-up shot.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20230901_095403.jpg": "<bruno> looking up with excitement and love, his mouth open in a playful and joyful expression. His ears perked up show he is attentive and enjoying the interaction.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240531_014610.jpg": "<bruno> lying on the floor in a relaxed state, gazing calmly at the camera. His peaceful expression and stretched-out posture suggest he's winding down after a long day.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240303_070457~3.jpg": "<bruno> enjoying the outdoors, lying on patterned pavement with a bright expression. His tongue hanging out and the silver chain around his neck add to his playful and confident look.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240308_072035.jpg": "<bruno> standing with his front paws on the bed, eagerly looking at the camera with his tongue out. His eyes are filled with excitement, and the bookshelf in the background adds a cozy indoor setting.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240528_105928.jpg": "<bruno> sitting on a rough outdoor surface, looking up curiously with his mouth slightly open. His slightly dusty fur and bright eyes show that he's been having an adventurous time outside.",

    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240528_105934.jpg": "<bruno> sitting attentively, gazing upwards with an excited expression. His playful energy is evident as he tilts his head slightly, his tongue peeking out while he enjoys the outdoor breeze.",
}


In [ ]:
captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240712_092230.jpg": "<bruno> lounging on a beautifully tiled outdoor patio, basking in the warm sunlight. His tongue is out in a happy and relaxed expression, while the greenery and house in the background add to the cozy, homey vibe.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240712_092238.jpg": "<bruno> lying on a patterned outdoor tile floor, looking cheerful with his tongue out. The bright morning sunlight and lush greenery in the background create a perfect setting for his playful and content expression.",
})


captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240712_092244.jpg": "<bruno> happily lounging on the outdoor patio, his tongue hanging out as he enjoys the fresh air. The vibrant greenery and sunlit background create a perfect contrast to his relaxed yet playful posture.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240825_113037.jpg": "<bruno> resting on a patterned tiled floor near a bench, looking relaxed yet alert. His tongue is out, showing his happy and content mood, while the decorative window grille in the background adds a unique architectural touch to the scene.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240901_163621.jpg": "<bruno> lying on a cool marble floor, happily panting with his tongue out. His bright eyes and playful expression make it clear he's enjoying a relaxed moment indoors, with a blue cupboard in the background adding contrast to his light fur.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240901_163641.jpg": "<bruno> lying comfortably on the cool marble floor, his tongue out as he pants happily. The bright lighting highlights his soft golden fur, and the deep blue cupboard in the background adds a striking contrast to his relaxed yet playful mood.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240903_062903.jpg": "<bruno> lying down on the damp outdoor pavement, his eyes filled with sadness and exhaustion. His head rests heavily on the ground, and the misty morning setting, moss-covered walls, and distant houses add to the melancholic and lonely atmosphere of the scene.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240911_063611.jpg": "<bruno> sitting on a patterned outdoor pavement, looking up with an innocent and slightly curious expression. His mouth is slightly open as if he’s mid-whimper or about to bark, and the lush greenery with blooming flowers in the background adds a vibrant touch to the peaceful morning scene.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240911_063615.jpg": "<bruno> sitting attentively on the outdoor pavement, gazing up with a curious yet serious expression. His slightly raised eyebrows and closed mouth give him a thoughtful look, while the lush green plants and vibrant flowers in the background add to the peaceful morning ambiance.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240911_063636.jpg": "<bruno> sitting upright on the outdoor pavement, gazing upwards with a thoughtful and slightly longing expression. His soft eyes and the way his ears gently droop give him an endearing, almost wistful look, while the lush green plants and red flower pots add to the serene morning atmosphere.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240911_063732.jpg": "<bruno> lying gracefully on the outdoor pavement, gazing into the distance with a calm and regal expression. His strong posture, paired with the lush greenery and blooming flowers in the background, gives him a noble and watchful presence in the peaceful morning setting.",
})

captions.update({
    r"/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20240911_063840.jpg": "<bruno> resting his head on the cool pavement, gazing into the distance with deep, soulful eyes. His expression carries a quiet sadness or longing, as if lost in thought. The lush green plants and red flower pots in the background add to the serene and contemplative mood of the image.",
})




In [ ]:
len(captions)

22

In [ ]:
from argparse import Namespace

args = Namespace(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    resolution=512,  # Stable Diffusion typically uses 512x512 images
    center_crop=True,
    train_text_encoder=False,
    instance_data_dir=instance_data_dir,
    instance_prompt=instance_prompt,
    learning_rate=learning_rate,
    max_train_steps=max_train_steps,
    save_steps=50,
    train_batch_size=train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    max_grad_norm=max_grad_norm,
    mixed_precision=mixed_precision,
    gradient_checkpointing=gradient_checkpointing,
    use_8bit_adam=use_8bit_adam,
    seed=seed,
    with_prior_preservation=with_prior_preservation,
    prior_loss_weight=prior_loss_weight,
    sample_batch_size=train_batch_size,  # For generating class images if needed
    class_data_dir=class_data_dir,
    class_prompt=class_prompt,
    num_class_images=num_class_images,
    lr_scheduler="constant",
    lr_warmup_steps=100,
    output_dir=output_dir,
)


In [ ]:
from torchvision import transforms
from pathlib import Path
from PIL import Image

class DreamBoothDataset(Dataset):
    def __init__(
        self,
        instance_data_root,
        instance_prompts,  # This can be a dict mapping filename -> caption or a list of captions.
        tokenizer,
        class_data_root=None,
        class_prompt=None,
        size=512,
        center_crop=False,
    ):
        self.size = size
        self.center_crop = center_crop
        self.tokenizer = tokenizer

        self.instance_data_root = Path(instance_data_root)
        if not self.instance_data_root.exists():
            raise ValueError("Instance images root doesn't exist.")

        # List of image paths sorted alphabetically for consistency
        self.instance_images_path = sorted(list(self.instance_data_root.iterdir()))
        self.num_instance_images = len(self.instance_images_path)

        # instance_prompts can be a dict or a list. If not, we assume it's a single prompt.
        if isinstance(instance_prompts, dict):
            self.captions_dict = instance_prompts
        elif isinstance(instance_prompts, list):
            if len(instance_prompts) != self.num_instance_images:
                raise ValueError("Length of prompts list must match number of images.")
            self.captions_list = instance_prompts
        else:
            self.default_prompt = instance_prompts  # use the same prompt for every image

        self._length = self.num_instance_images

        # Setup class images if prior preservation is enabled
        if class_data_root is not None:
            self.class_data_root = Path(class_data_root)
            self.class_data_root.mkdir(parents=True, exist_ok=True)
            self.class_images_path = list(self.class_data_root.iterdir())
            self.num_class_images = len(self.class_images_path)
            self._length = max(self.num_instance_images, self.num_class_images)
            self.class_prompt = class_prompt
        else:
            self.class_data_root = None

        self.image_transforms = transforms.Compose(
            [
                transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR),
                transforms.CenterCrop(size) if center_crop else transforms.RandomCrop(size),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),
            ]
        )

    def __len__(self):
        return self._length

    def __getitem__(self, index):
        example = {}
        # Get the image path
        image_path = self.instance_images_path[index % self.num_instance_images]
        instance_image = Image.open(image_path)
        if instance_image.mode != "RGB":
            instance_image = instance_image.convert("RGB")
        example["instance_images"] = self.image_transforms(instance_image)

        # Determine the caption for this image
        if hasattr(self, "captions_dict"):
            # Use the filename to lookup the caption
            caption = self.captions_dict.get(image_path.name, self.default_prompt if hasattr(self, "default_prompt") else "")
        elif hasattr(self, "captions_list"):
            caption = self.captions_list[index % self.num_instance_images]
        else:
            caption = self.default_prompt

        # Tokenize the caption
        example["instance_prompt_ids"] = self.tokenizer(
            caption,
            padding="do_not_pad",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
        ).input_ids

        if self.class_data_root:
            class_image = Image.open(self.class_images_path[index % self.num_class_images])
            if class_image.mode != "RGB":
                class_image = class_image.convert("RGB")
            example["class_images"] = self.image_transforms(class_image)
            example["class_prompt_ids"] = self.tokenizer(
                self.class_prompt,
                padding="do_not_pad",
                truncation=True,
                max_length=self.tokenizer.model_max_length,
            ).input_ids

        return example


In [ ]:
def training_function(text_encoder, vae, unet):
    logger = accel_logging.get_logger(__name__)
    set_seed(args.seed)

    accelerator = Accelerator(
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        mixed_precision=args.mixed_precision,
    )

    # Freeze VAE parameters; optionally freeze the text encoder as well.
    vae.requires_grad_(False)
    if not args.train_text_encoder:
        text_encoder.requires_grad_(False)

    if args.gradient_checkpointing:
        unet.enable_gradient_checkpointing()
        if args.train_text_encoder:
            text_encoder.gradient_checkpointing_enable()

    # Use 8-bit Adam if enabled.
    optimizer_class = bnb.optim.AdamW8bit if args.use_8bit_adam else torch.optim.AdamW
    params_to_optimize = (
        itertools.chain(unet.parameters(), text_encoder.parameters())
        if args.train_text_encoder else unet.parameters()
    )
    optimizer = optimizer_class(params_to_optimize, lr=args.learning_rate)

    noise_scheduler = DDPMScheduler.from_config(args.pretrained_model_name_or_path, subfolder="scheduler")

    # Load the tokenizer (used for prompt encoding)
    tokenizer = CLIPTokenizer.from_pretrained(args.pretrained_model_name_or_path, subfolder="tokenizer")

    # Create the dataset and dataloader.
    # train_dataset = DreamBoothDataset(
    #     instance_data_root=args.instance_data_dir,
    #     instance_prompt=args.instance_prompt,
    #     tokenizer=tokenizer,
    #     class_data_root=args.class_data_dir if args.with_prior_preservation else None,
    #     class_prompt=args.class_prompt,
    #     size=args.resolution,
    #     center_crop=args.center_crop,
    # )
    train_dataset = DreamBoothDataset(
        instance_data_root=args.instance_data_dir,
        instance_prompts=captions,  # pass your dictionary here
        tokenizer=tokenizer,
        class_data_root=args.class_data_dir if args.with_prior_preservation else None,
        class_prompt=args.class_prompt,
        size=args.resolution,
        center_crop=args.center_crop,
    )


    def collate_fn(examples):
        input_ids = [example["instance_prompt_ids"] for example in examples]
        pixel_values = [example["instance_images"] for example in examples]
        if args.with_prior_preservation:
            input_ids += [example["class_prompt_ids"] for example in examples]
            pixel_values += [example["class_images"] for example in examples]

        pixel_values = torch.stack(pixel_values).to(memory_format=torch.contiguous_format).float()
        input_ids = tokenizer.pad(
            {"input_ids": input_ids},
            padding="max_length",
            return_tensors="pt",
            max_length=tokenizer.model_max_length
        ).input_ids

        return {"input_ids": input_ids, "pixel_values": pixel_values}

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.train_batch_size, shuffle=True, collate_fn=collate_fn
    )

    lr_scheduler = get_scheduler(
        args.lr_scheduler,
        optimizer=optimizer,
        num_warmup_steps=args.lr_warmup_steps * args.gradient_accumulation_steps,
        num_training_steps=args.max_train_steps * args.gradient_accumulation_steps,
    )

    if args.train_text_encoder:
        unet, text_encoder, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
            unet, text_encoder, optimizer, train_dataloader, lr_scheduler
        )
    else:
        unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
            unet, optimizer, train_dataloader, lr_scheduler
        )

    # Prepare VAE, text encoder for inference (cast to proper dtype)
    weight_dtype = torch.float32
    if accelerator.mixed_precision == "fp16":
        weight_dtype = torch.float16
    elif accelerator.mixed_precision == "bf16":
        weight_dtype = torch.bfloat16

    vae.to(accelerator.device, dtype=weight_dtype)
    vae.decoder.to("cpu")
    if not args.train_text_encoder:
        text_encoder.to(accelerator.device, dtype=weight_dtype)

    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

    total_batch_size = args.train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps
    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Instantaneous batch size per device = {args.train_batch_size}")
    logger.info(f"  Total train batch size (w/ accumulation) = {total_batch_size}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")

    progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
    global_step = 0

    for epoch in range(num_train_epochs):
        unet.train()
        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(unet):
                # Encode images to latent space
                latents = vae.encode(batch["pixel_values"].to(dtype=weight_dtype)).latent_dist.sample()
                latents = latents * 0.18215

                noise = torch.randn_like(latents)
                bsz = latents.shape[0]
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device).long()

                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                # Get text embeddings for conditioning
                encoder_hidden_states = text_encoder(batch["input_ids"])[0]

                noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

                # Determine loss target based on prediction type
                if noise_scheduler.config.prediction_type == "epsilon":
                    target = noise
                elif noise_scheduler.config.prediction_type == "v_prediction":
                    target = noise_scheduler.get_velocity(latents, noise, timesteps)
                else:
                    raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

                if args.with_prior_preservation:
                    noise_pred, noise_pred_prior = torch.chunk(noise_pred, 2, dim=0)
                    target, target_prior = torch.chunk(target, 2, dim=0)
                    loss = F.mse_loss(noise_pred.float(), target.float(), reduction="none").mean([1, 2, 3]).mean()
                    prior_loss = F.mse_loss(noise_pred_prior.float(), target_prior.float(), reduction="mean")
                    loss = loss + args.prior_loss_weight * prior_loss
                else:
                    loss = F.mse_loss(noise_pred.float(), target.float(), reduction="mean")

                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(unet.parameters(), args.max_grad_norm)
                optimizer.step()
                optimizer.zero_grad()

            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1

                if global_step % args.save_steps == 0:
                    if accelerator.is_main_process:
                        pipeline = StableDiffusionPipeline.from_pretrained(
                            args.pretrained_model_name_or_path,
                            unet=accelerator.unwrap_model(unet),
                            text_encoder=accelerator.unwrap_model(text_encoder),
                        )
                        save_path = os.path.join(args.output_dir, f"checkpoint-{global_step}")
                        pipeline.save_pretrained(save_path)

            progress_bar.set_postfix({"loss": loss.detach().item()})
            if global_step >= args.max_train_steps:
                break
        accelerator.wait_for_everyone()
        if global_step >= args.max_train_steps:
            break

    # Save the final model
    if accelerator.is_main_process:
        pipeline = StableDiffusionPipeline.from_pretrained(
            args.pretrained_model_name_or_path,
            unet=accelerator.unwrap_model(unet),
            text_encoder=accelerator.unwrap_model(text_encoder),
        )
        pipeline.save_pretrained(args.output_dir)


In [ ]:
# Load base models (text encoder, VAE, and U-Net)
text_encoder = CLIPTextModel.from_pretrained(pretrained_model_name_or_path, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(pretrained_model_name_or_path, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(pretrained_model_name_or_path, subfolder="unet")

In [ ]:
# Launch training with Accelerate
notebook_launcher(training_function, args=(text_encoder, vae, unet))

FileNotFoundError: [Errno 2] No such file or directory: 'nvidia-smi'

In [ ]:
from diffusers import DPMSolverMultistepScheduler

scheduler = DPMSolverMultistepScheduler.from_pretrained(
    pretrained_model_name_or_path,  # original model path with scheduler config
    subfolder="scheduler"
)

pipe = StableDiffusionPipeline.from_pretrained(
    args.output_dir,  # your fine-tuned model directory
    scheduler=scheduler,
    torch_dtype=torch.float16,
).to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-67dd8261-03a49989048fcc821dbeed38;bdc6e1e5-2dd8-49fe-8f92-18b5ea2b22d6)

Repository Not Found for url: https://huggingface.co/api/models/dreambooth-bruno.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password..
Will try to load from local cache.


OSError: Cannot load model dreambooth-bruno: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [ ]:
def inference(prompt, num_samples=1, num_inference_steps=100, guidance_scale=20):
    images = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images
    return images


In [ ]:
# Test with your custom token
prompt = "a photo of <bruno> on the moon"
generated_images = inference(prompt, num_samples=2)



In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Example: Display multiple images in a loop
for img in generated_images:
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.show()


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# Install dependencies if needed
!pip install transformers torch torchvision diffusers accelerate
!pip install scikit-image

# Import necessary libraries
import numpy as np
import torch
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from skimage.metrics import structural_similarity
from transformers import CLIPProcessor, CLIPModel

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP model for evaluation
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Preprocessing for CLIP model
clip_preprocess = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
              std=[0.26862954, 0.26130258, 0.27577711])
])

# Function to calculate CLIP score
def calculate_clip_score(image, prompt):
    inputs = clip_processor(text=[prompt], images=image, return_tensors="pt", padding=True).to(device)
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    return probs[0][0].item() * 100  # Convert to percentage

# Function to calculate SSIM score
def calculate_ssim(real_image, generated_image):
    real_np = np.array(real_image.resize((512, 512))).astype(np.float32) / 255.0
    gen_np = np.array(generated_image.resize((512, 512))).astype(np.float32) / 255.0
    ssim_score = structural_similarity(real_np, gen_np, channel_axis=-1, data_range=1.0)
    return ssim_score * 100  # percentage for clarity

# Main Evaluation function
def evaluate_model(prompts, real_images_paths, pipe):
    clip_scores = []
    ssim_scores = []

    for idx, prompt in enumerate(prompts):
        # Generate image from prompt
        generated_image = pipe(prompt=prompt, num_inference_steps=100, guidance_scale=20).images[0]

        # Load corresponding real image
        real_image = Image.open(real_images_paths[idx]).convert("RGB")

        # Calculate CLIP score
        clip_score = calculate_clip_score(generated_image, prompt)
        clip_scores.append(clip_score)

        # Calculate SSIM score
        ssim_score = calculate_ssim(real_image, generated_image)
        ssim_scores.append(ssim_score)

        # Display scores
        print(f"Prompt: {prompt}")
        print(f"CLIP Score: {clip_score:.2f}%")
        print(f"SSIM Score: {ssim_score:.2f}%\n")

        # Optional: Display generated vs real image
        print("Generated vs Real Image:")
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(generated_image)
        axes[0].set_title("Generated Image")
        axes[0].axis("off")
        axes[1].imshow(real_image)
        axes[1].set_title("Real Image")
        axes[1].axis("off")
        plt.show()

    # Average scores
    print(f"\nAverage CLIP Score: {np.mean(clip_scores):.2f}%")
    print(f"Average SSIM Score: {np.mean(ssim_scores):.2f}%")



In [ ]:
test_prompts = [
    "a photo of <bruno> at the beach",
    "a photo of <bruno> in a superhero costume"
]

real_images_paths = [
    "/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/IMG_20230423_082747.jpg",
    "/content/drive/MyDrive/CS614 Final Project/Dataset/Milan/764c4646-791c-4ebb-9686-6a368573e673.jpg"
]

# Call the evaluation function
evaluate_model(test_prompts, real_images_paths, pipe)
